In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model

In [2]:
file=r'/Users/lalitsachan/Dropbox/DLV1/sentiment labelled sentences/imdb_labelled.txt'

In [3]:
imdb=pd.read_csv(file,sep='\t',header=None,names=['review','sentiment'])

In [ ]:
# imdb

In [4]:
imdb_train,imdb_test=train_test_split(imdb,test_size=0.2,random_state=2)

In [5]:
x_train=imdb_train['review']
y_train=imdb_train['sentiment']
x_test=imdb_test['review']
y_test=imdb_test['sentiment']

In [6]:
sent_lens=[]
for sent in imdb_train['review']:
    sent_lens.append(len(word_tokenize(sent)))
    
    

In [7]:
max(sent_lens)

1616

In [8]:
# sent_lens

In [9]:
np.quantile(sent_lens,0.95)


40.0

In [10]:
# We can see that 95% review text are of lengths less than or equal to 37. We'll keep the max length to 30

In [11]:
max_len = 40

tok = Tokenizer(char_level=False,split=' ')

tok.fit_on_texts(x_train)

In [12]:
tok.index_word

{1: 'the',
 2: 'a',
 3: 'and',
 4: 'of',
 5: 'is',
 6: 'this',
 7: 'it',
 8: 'i',
 9: 'to',
 10: 'was',
 11: 'movie',
 12: 'in',
 13: 'film',
 14: 'that',
 15: '1',
 16: '0',
 17: 'but',
 18: 'for',
 19: 'as',
 20: 'with',
 21: 'are',
 22: 'on',
 23: 'not',
 24: 'you',
 25: 'one',
 26: 'very',
 27: 'bad',
 28: 'just',
 29: 'so',
 30: 'good',
 31: 'all',
 32: 'an',
 33: "it's",
 34: 'there',
 35: 'be',
 36: 'by',
 37: 'about',
 38: 'at',
 39: 'if',
 40: 'out',
 41: 'great',
 42: 'his',
 43: 'from',
 44: 'like',
 45: 'have',
 46: 'time',
 47: 'were',
 48: 'well',
 49: 'has',
 50: 'even',
 51: 'really',
 52: 'my',
 53: 'or',
 54: 'who',
 55: 'acting',
 56: 'he',
 57: 'when',
 58: 'most',
 59: 'see',
 60: 'how',
 61: 'more',
 62: 'characters',
 63: 'would',
 64: 'no',
 65: 'only',
 66: 'ever',
 67: 'made',
 68: 'also',
 69: 'best',
 70: '10',
 71: 'plot',
 72: 'some',
 73: 'your',
 74: 'do',
 75: 'its',
 76: 'character',
 77: 'real',
 78: 'because',
 79: 'love',
 80: "didn't",
 81: 'movies

In [14]:
sequences_train = tok.texts_to_sequences(x_train)

In [15]:
# sequences_train

In [17]:
vocab_len=len(tok.index_word.keys())

In [18]:
vocab_len

2688

In [19]:
sequences_matrix_train = sequence.pad_sequences(sequences_train,maxlen=max_len)

In [20]:
sequences_matrix_train

array([[  0,   0,   0, ...,  30,  90, 426],
       [  0,   0,   0, ...,  10, 275,  94],
       [  0,   0,   0, ...,  18,   1,  71],
       ...,
       [  0,   0,   0, ...,  44,  64,  84],
       [  0,   0,   0, ..., 284, 284, 284],
       [  0,   0,   0, ..., 383,  20, 307]], dtype=int32)

In [21]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    
    layer = Embedding(vocab_len+1,500,input_length=max_len,
                      mask_zero=True)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model



In [22]:
model = RNN()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 40)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 40, 500)           1344500   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                144640    
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257   

In [23]:
model.compile(loss='binary_crossentropy',optimizer='adam',
              metrics=['accuracy'])

In [24]:
sequences_test = tok.texts_to_sequences(x_test)
sequences_matrix_test = sequence.pad_sequences(sequences_test,
                                               maxlen=max_len)

In [27]:
model.fit(sequences_matrix_train,y_train.values,batch_size=50,
          epochs=50,validation_data=(sequences_matrix_test,y_test.values))

Train on 598 samples, validate on 150 samples
Epoch 1/50
598/598 [==============================] - 7s 11ms/sample - loss: 0.6925 - accuracy: 0.5268 - val_loss: 0.6869 - val_accuracy: 0.7067
Epoch 2/50
598/598 [==============================] - 1s 2ms/sample - loss: 0.6616 - accuracy: 0.8144 - val_loss: 0.6282 - val_accuracy: 0.7800
Epoch 3/50
598/598 [==============================] - 1s 2ms/sample - loss: 0.4351 - accuracy: 0.8946 - val_loss: 0.5102 - val_accuracy: 0.7600
Epoch 4/50
598/598 [==============================] - 1s 2ms/sample - loss: 0.1478 - accuracy: 0.9632 - val_loss: 0.5742 - val_accuracy: 0.8000
Epoch 5/50
598/598 [==============================] - 1s 2ms/sample - loss: 0.0526 - accuracy: 0.9883 - val_loss: 0.6487 - val_accuracy: 0.7933
Epoch 6/50
598/598 [==============================] - 1s 2ms/sample - loss: 0.0246 - accuracy: 0.9916 - val_loss: 0.8310 - val_accuracy: 0.7867
Epoch 7/50
598/598 [==============================] - 1s 2ms/sample - loss: 0.0086 - accu

In [28]:
predictions=model.predict(sequences_matrix_test)

In [29]:
from sklearn.metrics import roc_auc_score

In [30]:
roc_auc_score(y_test,predictions)

0.8350331007335838

In [31]:
# if you notice the validation loss , it is actually increasing after some epochs , best model appears much earlier than 
# 50th epoch